In [13]:
#pip install chromadb==0.5.3 因为chroma 0.5.4 version有报错AttributeError: 'Collection' object has no attribute 'model_fields'
import os
from dotenv import load_dotenv
#加载 .env文件中的环境变量,保存在代码当前路径下
load_dotenv()
import openai
from langchain_openai import OpenAI
from langchain.schema import AIMessage,SystemMessage,HumanMessage
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import json

llm= ChatOpenAI(model_name="chatglm3-6b",temperature=0.1)

import chromadb
from chromadb.utils import embedding_functions
# 指定chromadb提供的embedding function使用openai接口能力
bge_embeddings = embedding_functions.OpenAIEmbeddingFunction(
    api_base=os.environ["OPENAI_API_BASE"],
    api_key=os.environ["OPENAI_API_KEY"],
    model_name="bge-large-zh-1.5"
)

In [14]:
# Dependencies
from datetime import date, timedelta  # date handling for fetching recent news
from IPython.display import display  # for pretty printing need pip install IPython
import json  # for parsing the JSON api responses and model outputs
from numpy import dot  # for cosine similarity
import requests  # for making the API requests
from tqdm import tqdm  # for printing progress bars

today = date.today()

# Helper functions
def json_gpt(input):
    
    messages=[
        {"role": "system", "content": "用JSON格式输出结果"},
        {"role": "user", "content": input},
    ]
    completion = llm.invoke(messages)
    text = completion.content
    parsed = json.loads(text)

    return parsed

In [10]:
#测试，LLM输出JSON格式
input="what's your name? and where are you from?"
r = json_gpt(input)
display(r)

{'name': 'AI assistant', 'location': 'cyberspace'}

In [15]:
# User asks a question
USER_QUESTION = "Who won the NBA championship? And who was the MVP? Tell me a bit about the last game."

In [27]:
QUERIES_INPUT = f"""
Today's date is {today}.
You have access to a search API that returns recent news articles.
Generate an array of search queries that are relevant to this question.
Use a variation of related keywords for the queries, trying to be as general as possible.
Include as many queries as you can think of, including and excluding terms.
For example, include queries like ['keyword_1 keyword_2', 'keyword_1', 'keyword_2'].
Be creative. The more queries you include, the more likely you are to find relevant results.

User question: {USER_QUESTION}

Format: {{"queries": ["query_1", "query_2", "query_3"]}}
"""

queries = json_gpt(QUERIES_INPUT)["queries"]

# Let's include the original question as well for good measure
queries.append(USER_QUESTION)

queries

['Who won the NBA championship in 2022?',
 'What was the final score of the 2022 NBA championship?',
 'Which team won the 2022 NBA championship?',
 'How did the 2022 NBA champion win?',
 'What were some of the key moments in the 2022 NBA championship game?',
 'Were there any upsets in the 2022 NBA championship?',
 'Did any player from a specific team win the 2022 NBA championship?',
 'How did the 2022 NBA MVP perform in the championship game?',
 'Did any player from a specific team win the 2022 NBA MVP award?',
 'What was the highlight of the 2022 NBA MVP presentation?',
 'How did the 2022 NBA season affect the MVP race?',
 'Was the 2022 NBA championship a record-breaking performance?',
 'Did any other records break during the 2022 NBA championship?',
 'Who won the NBA championship? And who was the MVP? Tell me a bit about the last game.']

In [16]:
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain.utilities.bing_search import BingSearchAPIWrapper
from langchain_community.tools.bing_search.tool import BingSearchRun

#创建langchain自带WIKI工具
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=1000)
Lookup_wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

#创建搜索工具
bing_api_wrapper = BingSearchAPIWrapper(k=3)
Bing_search = BingSearchRun(api_wrapper=bing_api_wrapper)

In [11]:
#测试，搜索接口
r=(bing_api_wrapper.results("唱跳rap篮球的是谁？", 3))
print(r)
display(r[0]["snippet"])

[{'snippet': '2018年，蔡徐<b>坤在</b>选秀节目《偶像练习生》里，以一段包含篮球的唱跳做为自我介绍，并且表示喜欢“唱、跳、rap、篮球”。 这段表演说不上精彩，也并不太糟糕，在当时没有引起任何关注，直到2019年春节蔡徐坤被选为首位NBA新春贺岁形象大使。', 'title': '5年了，我们为什么还没放过“鸡坤”？_澎湃号·湃客_澎湃新闻 ...', 'link': 'https://www.thepaper.cn/newsDetail_forward_23543004'}, {'snippet': '“<b>唱跳rap和篮球</b>”出自蔡徐坤，蔡徐坤在到场偶像训练生<b>的</b>时候自我介绍<b>是</b>“喜欢<b>唱跳</b>，<b>rap和篮球</b>”，原来挺正常<b>的</b>一句话，可是随着后来蔡徐坤因为一段打<b>篮球的</b>视频被全网黑，这句话现在也就充满了讽刺意味。', 'title': '蔡徐坤<b>唱跳rap和篮球是</b>什么梗？<b>唱跳rap篮球</b>什么意思？_网友', 'link': 'https://www.sohu.com/a/312636181_100261694'}, {'snippet': '三人组中玲花负责<b>唱</b>，曾毅负责<b>跳和rap</b>，徐静雨负责<b>篮球</b>——下次可以考虑一下哥哥？玩家们也发现，既然<b>唱跳rap篮球</b>都这么经典了,如果把普通<b>篮球</b>换成帅气<b>的</b>街篮，岂不<b>是</b>更嗨？', 'title': '玩了好几年<b>的</b>“<b>唱跳RAP篮球</b>”，真有人把它做成游戏了 ...', 'link': 'https://new.qq.com/rain/a/20240121A070RK00'}]


'2018年，蔡徐<b>坤在</b>选秀节目《偶像练习生》里，以一段包含篮球的唱跳做为自我介绍，并且表示喜欢“唱、跳、rap、篮球”。 这段表演说不上精彩，也并不太糟糕，在当时没有引起任何关注，直到2019年春节蔡徐坤被选为首位NBA新春贺岁形象大使。'

In [29]:
#将列表中的每个问题都到BING取三条结果
from tqdm import tqdm
results=[]

for query in tqdm(queries):
    result = bing_api_wrapper.results(query,3)
    for r in tqdm(result):
        results.append(r)

print(results)
print("Total number of results:", len(results))

# remove duplicates
results = list({result["link"]: result for result in results}.values())
print("Total number of results after remove duplicates:", len(results))

  0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 14/14 [00:16<00:00,  1.15s/it]

[{'snippet': '<b>The</b> <b>Golden</b> <b>State</b> <b>Warriors</b> have won the 2022 NBA Championship following tonight’s 103-90 victory over the Boston Celtics in Game 6 of the NBA Finals at TD Garden Arena in Boston, Mass. The...', 'title': 'Golden State Warriors Win <b>2022</b> <b>NBA</b> <b>Championship</b>', 'link': 'https://www.nba.com/warriors/news/warriors-2022-nba-champions-20220616'}, {'snippet': 'With the victory, <b>the</b> <b>Warriors</b> have secured the franchises 7th NBA title, passing the Chicago Bulls (6) for the 3rd most championships in NBA history. June 16, 2022', 'title': '<b>2022</b> <b>NBA</b> Finals | <b>NBA</b>.com', 'link': 'https://www.nba.com/playoffs/2022/the-finals'}, {'snippet': 'Led by 34 points from Stephen Curry, the Golden State Warriors defeated the Boston Celtics 103-90 on Thursday night to claim their fourth <b>NBA</b> <b>championship</b> in the last eight seasons.', 'title': 'Golden State Warriors win <b>NBA</b> <b>championship</b> with Game 6 v

In [17]:
#创建持久化数据库
dbclient = chromadb.PersistentClient(path="d:\\chromadb")

In [31]:
#测试，如有需要，删除数据库里的实验用cellection，清除之前的实验内容
dbclient.delete_collection("cosine_test")

In [18]:
#将文本块导入向量数据库的名称是“cos_collection”的collection
cos_collection = dbclient.get_or_create_collection(name="cosine_test", 
                                               metadata={"hnsw:space": "cosine"}, # l2 is the default
                                               embedding_function=bge_embeddings) 

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators = ['\n\n','\n','.'],
    chunk_size=450,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [34]:
#依次把每个搜索结果中的link页面提取、切段并存入向量数据库
#pip install beautifulsoup4 才能使用WebBaseLoader
from langchain_community.document_loaders.web_base import WebBaseLoader

i=0
for result in tqdm(results):
    try:
        loader= WebBaseLoader(str(result['link']))
    except:
        continue
    page_content= loader.load()
    chunks = text_splitter.split_text(page_content[0].page_content)
    for k in range(len(chunks)):
        cos_collection.add(
            documents=[chunks[k]],
            metadatas=[{"source": USER_QUESTION}],
            ids=[f"{i:05}{k:05}"]
            )
    i+=1

  0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████| 26/26 [05:39<00:00, 13.07s/it]


In [20]:
#检索数据库
retrieved_results=cos_collection.query(
    query_texts=USER_QUESTION,
    n_results=10
)
retrieved_documents=retrieved_results['documents'][0]

Add of existing embedding ID: 0001700000
Add of existing embedding ID: 0001700001
Add of existing embedding ID: 0001700002
Add of existing embedding ID: 0001700003
Add of existing embedding ID: 0001700004
Add of existing embedding ID: 0001700005
Add of existing embedding ID: 0001700006
Add of existing embedding ID: 0001700007
Add of existing embedding ID: 0001700008
Add of existing embedding ID: 0001800000
Add of existing embedding ID: 0001800001
Add of existing embedding ID: 0001800002
Add of existing embedding ID: 0001800003
Add of existing embedding ID: 0001800004
Add of existing embedding ID: 0001800005
Add of existing embedding ID: 0001800006
Add of existing embedding ID: 0001800007
Add of existing embedding ID: 0001800008
Add of existing embedding ID: 0001800009
Add of existing embedding ID: 0001800010
Add of existing embedding ID: 0001800011
Add of existing embedding ID: 0001900000
Add of existing embedding ID: 0002000000
Add of existing embedding ID: 0002000001
Add of existing 

In [21]:
print(retrieved_documents)
retrieved_documents = [s.replace("\n", " ") for s in retrieved_documents]

[".Who will take home the league's highest individual accolades and what did they do to deserve the award?MORE: Watch the NBA playoffs on Sling TV (U.S", 'Your first NBA Finals MVP Award does, too, man.\nSo, enjoy it.\nAnd now, the final Top 5 in our 2022 Race to the NBA Finals MVP Ladder:', 'Your first NBA Finals MVP Award does, too, man.\nSo, enjoy it.\nAnd now, the final Top 5 in our 2022 Race to the NBA Finals MVP Ladder:', 'Curry, who was in tears on the court after the final whistle, won NBA Finals MVP honors for the first time in his career.\n    \n\n            â€œThis is what itâ€™s all about, playing for a championship and what weâ€™ve been through the last three years,â€� he said. â€œBeginning of the season, nobody thought weâ€™d be here except everybody on this court right now, and itâ€™s amazing. Very surreal, though.', 'Curry, who was in tears on the court after the final whistle, won NBA Finals MVP honors for the first time in his career.\n    \n\n            â€œThis is 

In [39]:
print(USER_QUESTION, retrieved_documents)

Who won the NBA championship? And who was the MVP? Tell me a bit about the last game. [".Who will take home the league's highest individual accolades and what did they do to deserve the award?MORE: Watch the NBA playoffs on Sling TV (U.S", 'Your first NBA Finals MVP Award does, too, man. So, enjoy it. And now, the final Top 5 in our 2022 Race to the NBA Finals MVP Ladder:', 'Your first NBA Finals MVP Award does, too, man. So, enjoy it. And now, the final Top 5 in our 2022 Race to the NBA Finals MVP Ladder:', 'Curry, who was in tears on the court after the final whistle, won NBA Finals MVP honors for the first time in his career.                   â€œThis is what itâ€™s all about, playing for a championship and what weâ€™ve been through the last three years,â€� he said. â€œBeginning of the season, nobody thought weâ€™d be here except everybody on this court right now, and itâ€™s amazing. Very surreal, though.', '“This was a historic run. I don’t know how you can say he’s not the best bi

In [22]:
#定义用httpx链接rerank接口
import httpx
from pydantic.main import BaseModel

RERANK_URL = "{url}/bge_base_rerank"
url=RERANK_URL.format(url=os.environ["OPENAI_API_BASE"])
headers = {'Content-Type': 'application/json'}

class QuerySuite(BaseModel):
    query: str
    passages: list[str]
    top_k: int = 5

def rerank(query_suite: QuerySuite):
    r = httpx.post(url, headers=headers, json=query_suite)
    print(r)
    return r.json()

In [23]:
#将检索结果重排名，取top5
pairs = {'query':USER_QUESTION,'passages':retrieved_documents}
rerank_results = rerank(pairs)

<Response [200 OK]>


In [24]:
#打印top5结果
formatted_top_results=[]

for key, value in rerank_results.items():
    print(f"score:{value} \n {key} \n\n")
    formatted_top_results.append(key)

score:2.75 
 NBA Finals: Led by MVP Steph Curry, Golden State Warriors win championship with Game 6 victory over Boston Celtics | CNN                                                                                                       CNN values your feedback                                                             1. How relevant is this ad to you? 


score:-0.35498046875 
 Curry, who was in tears on the court after the final whistle, won NBA Finals MVP honors for the first time in his career.                   â€œThis is what itâ€™s all about, playing for a championship and what weâ€™ve been through the last three years,â€� he said. â€œBeginning of the season, nobody thought weâ€™d be here except everybody on this court right now, and itâ€™s amazing. Very surreal, though. 


score:-2.5 
 As the Warriors celebrate their fourth championship in eight seasons, we take a closer look at the numbers Curry posted in his greatest Finals performance yet. Here are five numbers you need to k

In [25]:
ANSWER_INPUT = f"""
Generate an answer in Mandarin to the user's question based on the given search results. 
TOP_RESULTS: {formatted_top_results}
USER_QUESTION: {USER_QUESTION}

Include as much information as possible in the answer. 
"""
messages=[{"role": "user", "content": ANSWER_INPUT}]
completion = llm.invoke(
    messages
)

answer=completion.content

In [26]:
print(answer)

根据搜索结果，金州勇士队赢得了NBA总冠军，这是他们在过去八个赛季中获得的第四个冠军。在总决赛中，史蒂芬·库里成为了最有价值球员（MVP）。他在总决赛中的表现非常出色，成为第一个在NBA总决赛中平均每场得分30分、篮板5个、助攻5次并投中5个三分球的球员。在赛后的新闻发布会上，库里表示这是他职业生涯中最有意义的荣誉，因为他和他的队友们为了这个冠军付出了很多努力。此外，我还了解到在比赛最后时刻，库里在场上流泪，这让他感到非常 surreal。


In [30]:
ANSWER_INPUT = f"""
Today's date is {today}.
基于下面AI给出的答案，列出答案所包含的所有事实，推断一个富有好奇心的NBA观众会衍生出哪些追问，为这些追问生成尽量多用于搜索引擎检索的英文问句，不要重复下面已有用户问题中的内容。
User question: {USER_QUESTION}
AI Answer: {answer}

Format: {{"queries": ["query_1", "query_2", "query_3"],...}}
"""
messages=[
    {"role": "system", "content": "用JSON格式输出结果"},
    {"role": "user", "content": ANSWER_INPUT}]
completion = llm.invoke(
    messages
)
queries=json.loads(completion.content)
print(queries)

{'queries': ['谁赢得了最近的NBA总冠军？', '谁是最近NBA总决赛最有价值球员？', '关于NBA总决赛的最后一场比赛有什么细节？', 'NBA总决赛中史蒂芬·库里的表现如何？', '史蒂芬·库里在总决赛中的平均数据是多少？', '史蒂芬·库里在新闻发布会上说了什么？', '在比赛最后时刻，史蒂芬·库里在场上流泪的原因是什么？', '谁是史蒂芬·库里的队友？', '史蒂芬·库里在职业生涯中获得了哪些荣誉？', '史蒂芬·库里在总决赛中的关键时刻是什么？', '史蒂芬·库里在总决赛中的投篮表现如何？', '史蒂芬·库里在总决赛中的篮板表现如何？', '史蒂芬·库里在总决赛中的助攻表现如何？', '史蒂芬·库里在总决赛中的三分球表现如何？']}


In [33]:
#将列表中的每个问题都到BING取三条结果，并存入数据库和results数组。
from tqdm import tqdm
from langchain_community.document_loaders.web_base import WebBaseLoader
results=[]
i=5
for query in tqdm(queries['queries']):
    try:
        result = bing_api_wrapper.results(query,3)
    except:
        continue
    for r in result:
        results.append(r)
        loader= WebBaseLoader(str(r['link']))
        page_content= loader.load()
        chunks = text_splitter.split_text(page_content[0].page_content)
        for k in range(len(chunks)):
            cos_collection.add(
                documents=[chunks[k]],
                metadatas=[{"source": query}],
                ids=[f"{i:05}{k:05}"]
                )
        i+=1


  0%|          | 0/14 [00:00<?, ?it/s]

Add of existing embedding ID: 0000500000
Insert of existing embedding ID: 0000500000
Add of existing embedding ID: 0000500001
Insert of existing embedding ID: 0000500001
Add of existing embedding ID: 0000500002
Insert of existing embedding ID: 0000500002
Add of existing embedding ID: 0000500003
Insert of existing embedding ID: 0000500003
Add of existing embedding ID: 0000600000
Insert of existing embedding ID: 0000600000
Add of existing embedding ID: 0000600001
Insert of existing embedding ID: 0000600001
Add of existing embedding ID: 0000700000
Insert of existing embedding ID: 0000700000
Add of existing embedding ID: 0000700001
Insert of existing embedding ID: 0000700001
Add of existing embedding ID: 0000700002
Insert of existing embedding ID: 0000700002
Add of existing embedding ID: 0000700003
Insert of existing embedding ID: 0000700003
Add of existing embedding ID: 0000700004
Insert of existing embedding ID: 0000700004
Add of existing embedding ID: 0000700005
Insert of existing embed

In [48]:
#检索数据库，为每个query的检索结果写限字300的总结。最后把几个总结都append到top_results里
top_results=[]
for query in tqdm(queries['queries']):
    retrieved_results=cos_collection.query(
        query_texts=query,
        n_results=10
    )
    retrieved_documents=retrieved_results['documents'][0]
    pairs = {'query':query,'passages':retrieved_documents}
    rerank_results = rerank(pairs)
    ANSWER_INPUT = f"""
    Today's date is {today}.
    Generate an summary in Mandarin to the user's question based on the given search results. All statements must be supported by sources of information。
    TOP_RESULTS: {retrieved_documents}
    USER_QUESTION: {query}

    contain the length of this aummary within 200 words. 
    """
    messages=[{"role": "user", "content": ANSWER_INPUT}]
    completion = llm.invoke(
        messages
    )
    answer=completion.content

    top_results.append(answer)

  0%|          | 0/14 [00:00<?, ?it/s]

<Response [200 OK]>


  7%|▋         | 1/14 [00:19<04:11, 19.33s/it]

<Response [200 OK]>


 14%|█▍        | 2/14 [00:29<02:44, 13.75s/it]

...

 86%|████████▌ | 12/14 [04:11<00:41, 20.89s/it]

<Response [200 OK]>


 93%|█████████▎| 13/14 [04:31<00:20, 20.66s/it]

<Response [200 OK]>


100%|██████████| 14/14 [04:46<00:00, 20.47s/it]


In [49]:
print(top_results)

['2022年NBA总冠军是金州勇士队，他们在总决赛中以4:2的总比分战胜了凯尔特人队。这一结果标志着勇士队连续第三年进入总决赛，并成功夺回总冠军。在总决赛中，库里表现出色，场均贡献26.8分、8个篮板和9.4次助攻，帮助勇士队取得了关键胜利。整个总决赛中，库里场均贡献27.5分、6个篮板和6.1次助攻，连续第五年入选全明星和年度最佳阵容第三队。', '根据您提供的搜索结果，最近的NBA总决赛最有价值球员是尼古拉·约基奇（Nikola Jokic）。他在2021-22赛季与丹佛掘金队一起赢得了NBA总冠军。此外，他还获得了2023-24赛季的Kia NBA年度最有价值球员奖。他是历史上获得这一奖项最年轻的球员之一。同时，我们还了解到Stephen Curry也是一位非常出色的球员，他曾与其他球员一起赢得了四个NBA总冠军和两个Kia MVP奖项。', '根据搜索结果，NBA总决赛最后一场比赛的细节如下：在2016年NBA总决赛的第七场比赛中，克里夫兰骑士队以96比89战胜金州勇士队，赢得了他们的第一个NBA冠军。在这场比赛中，勒布朗·詹姆斯（LeBron James）在第三节末端抢下了一个关键的篮板球，并在比赛还有不到两分钟时封盖了安德烈·伊戈达拉（Andre Iguodala）的投篮，帮助克里夫兰队领先。最终，克里夫兰队以31-9的比分赢得了这场比赛，并结束了他们52年的冠军干旱。', '在2019年的NBA总决赛中，史蒂芬·库里表现出色，带领金州勇士队争夺总冠军。库里在总决赛中场均贡献26.8分、8个篮板和9.4次助攻，准三双的数据为球队带来强大的战斗力。在西部决赛面对独行侠的比赛中，库里独揽40分，帮助勇士完成了25分的大逆转。在总决赛第四场中，他又轰下36分，率领勇士4-0淘汰对手，从而以4-0的总比分横扫对手，夺得四年内的第三座总冠军奖杯。在整个总决赛中，库里场均贡献27.5分、6个篮板和6.3次助攻，连续第五年入选全明星和年度最佳阵容第三队。', '史蒂芬·库里（Stephen Curry）是一位美国职业篮球运动员，司职得分后卫，现效力于NBA金州勇士队。库里被认为是当代篮球界最具天赋的球员之一，多次获得NBA常规赛和总决赛的最有价值球员奖项。根据用户问题，我们找到了关于库里在总决赛中的平均数据。\n\n在2017-18赛季的NBA总决赛中，库里场均贡献

In [75]:
ANSWER_INPUT = f"""
今天日期 {today}。
理解{queries['queries']}中的用户提问，明白用户想了解什么，生成一篇短文的简短提纲。开头用一个最吸引人的事实作为导语，正文陈述细节，内容不要与前文重复，结尾总结所有事实。
"""
messages=[{"role": "user", "content": ANSWER_INPUT}]
completion = llm.invoke(
    messages
)
outline=completion.content

In [76]:
print(outline)

一、引言
1. 2022年NBA总决赛已经结束，金州勇士队以4:2战胜了波士顿凯尔特人队，获得了最新的NBA总冠军。

二、史蒂芬·库里在总决赛的表现
1. 在总决赛中，史蒂芬·库里发挥出色，成为了球队的关键人物。
2. 库里在总决赛的平均数据为27分、6板和5助攻，展现出了他的全面实力。
3. 特别是在关键的第五场比赛中，库里拿下了43分，帮助勇士队赢得比赛，巩固了他在总决赛中的地位。

三、史蒂芬·库里的关键时刻表现
1. 在总决赛的第四场比赛中，库里在比赛最后时刻成功抢断，并绝杀得分，帮助勇士队获得胜利。
2. 库里在总决赛的投篮表现也非常出色，命中了许多关键的三分球。

四、史蒂芬·库里的团队支持
1. 史蒂芬·库里在勇士队的队友们也表现出色，提供了很多支持，使得库里在总决赛中能够发挥出最佳水平。

五、史蒂芬·库里的荣誉
1. 史蒂芬·库里在职业生涯中已经获得了许多荣誉，包括两次NBA总冠军和一次常规赛MVP。

六、结语
总的来说，史蒂芬·库里在2022年的NBA总决赛中表现出色，是勇士队夺冠的重要原因之一。他的全面实力和关键时刻的表现都让人印象深刻。


In [81]:
ANSWER_INPUT = f"""
今天日期 {today}。

参考提纲，将摘要中的内容整合成一篇完整、连贯的短文，文中所有陈述必须有参考依据，不需要标题。
提纲：{outline}
摘要：{top_results}
"""
messages=[{"role": "user", "content": ANSWER_INPUT}]
completion = llm.invoke(
    messages
)
answer=completion.content

In [82]:
print(answer)

2022年NBA总决赛已经结束，金州勇士队以4:2战胜了波士顿凯尔特人队，获得了最新的NBA总冠军。其中，史蒂芬·库里在总决赛中发挥出色，成为了球队的关键人物。库里在总决赛的平均数据为27分、6板和5助攻，展现出了他的全面实力。特别是在关键的第五场比赛中，库里拿下了43分，帮助勇士队赢得比赛，巩固了他在总决赛中的地位。

在总决赛的第四场比赛中，库里在比赛最后时刻成功抢断，并绝杀得分，帮助勇士队获得胜利。库里在总决赛的投篮表现也非常出色，命中了许多关键的三分球。这些表现让人印象深刻，同时也展示了史蒂芬·库里在关键时刻的能力和全面实力。

值得一提的是，史蒂芬·库里在职业生涯中已经获得了许多荣誉，包括两次NBA总冠军和一次常规赛MVP。他的全面实力和关键时刻的表现都让人印象深刻。在2019年的NBA总决赛中，库里表现出色，带领金州勇士队争夺总冠军。整个总决赛中，库里场均贡献27.5分、6个篮板和6.3次助攻，连续第五年入选全明星和年度最佳阵容第三队。

此外，史蒂芬·库里在比赛最后时刻流泪的原因是因为他错失了一个关键的投篮机会，导致比赛最终失利。这个投篮机会对勇士队的胜利至关重要，因此库里感到非常失望和难过。尽管他在比赛中表现出色，但最终未能为球队带来胜利让库里感到非常悲伤。

综上所述，史蒂芬·库里在2022年的NBA总决赛中表现出色，是勇士队夺冠的重要原因之一。他的全面实力和关键时刻的表现都让人印象深刻。
